In [18]:
import sagemaker
import boto3
from sagemaker import get_execution_role
import pandas as pd

In [2]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))
sagemaker_session = sagemaker.Session()

AWS Region: ap-south-1


In [3]:
# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [4]:
role

'arn:aws:iam::565927539742:role/service-role/AmazonSageMaker-ExecutionRole-20220525T190861'

Upload the data for training

In [5]:
train_input = sagemaker_session.upload_data("data")

In [6]:
train_input

's3://sagemaker-ap-south-1-565927539742/data'

##Create SageMaker Scikit Estimator

In [7]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'water_prediction.py'

sklearn = SKLearn(
    entry_point=script_path,
    instance_type="ml.m4.xlarge",
    framework_version="0.20.0",
    py_version="py3",
    role=role,
    sagemaker_session=sagemaker_session)

Train SKLearn Estimator on Startup data

In [8]:
sklearn.fit({'train': train_input})

2022-06-10 14:58:42 Starting - Starting the training job...
2022-06-10 14:59:08 Starting - Preparing the instances for trainingProfilerReport-1654873122: InProgress
......
2022-06-10 15:00:11 Downloading - Downloading input data......
2022-06-10 15:01:06 Training - Downloading the training image...
2022-06-10 15:01:37 Training - Training image download completed. Training in progress..2022-06-10 15:01:40,342 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-06-10 15:01:40,345 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 15:01:40,356 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-06-10 15:01:40,832 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 15:01:40,851 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 15:01:40,870 sagemaker-training-toolkit INFO     No GPUs detected (no

Deploy the model

In [9]:
deployment = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

------!

In [10]:
deployment.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-scikit-learn-2022-06-10-15-02-24-684'

In [12]:
import pandas as pd
from matplotlib import pyplot as plt
client = boto3.client('iotanalytics')
dataset = 'main_channel_dataset'
dataset_url = client.get_dataset_content(datasetName = dataset)['entries'][0]['dataURI']

In [14]:
df = pd.read_csv(dataset_url)

In [17]:
df

,turbidity,__dt
0,2.421875,2022-06-09 00:00:00.000
1,2.421875,2022-06-09 00:00:00.000
2,2.421875,2022-06-09 00:00:00.000
3,2.421875,2022-06-09 00:00:00.000
4,2.421875,2022-06-09 00:00:00.000
...,...,...
557,2.421875,2022-06-09 00:00:00.000
558,2.421875,2022-06-09 00:00:00.000
559,2.421875,2022-06-09 00:00:00.000
560,2.421875,2022-06-09 00:00:00.000


In [15]:
x = df.iloc[0][0]

In [16]:
deployment.predict([[8.3, 22018, x]])

array([0.])